<a href="https://colab.research.google.com/github/omar-sharif03/NAACL-SRW-2021/blob/main/ML_Algorithms_NAACL_(Emotion_Data).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%time
import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns
import re
import nltk
import json
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers
from keras import models
from keras import layers
from tensorflow.keras.layers import LSTM,GRU
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report 
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score
from sklearn.metrics import average_precision_score,roc_auc_score, roc_curve, precision_recall_curve
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from tensorflow.keras.preprocessing.text import Tokenizer
np.random.seed(42)

CPU times: user 3.17 s, sys: 475 ms, total: 3.64 s
Wall time: 4.45 s


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Reading  data

In [ ]:
#read Data
data =pd.read_excel('/content/drive/MyDrive/Thesis/Shared/DFBA Train Data.xlsx')
data_df = data[:][0:6478]
text_data = data_df["text"]
labels = data_df["label2"]
classes = data_df['class2']


/usr/local/lib/python3.7/dist-packages/openpyxl/worksheet/_reader.py:211: UserWarning: Cell E4661 is marked as a date but the serial value 6691519.586805556 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)


In [ ]:
labels.value_counts()

no          2746
moderate    1670
severe      1031
mild        1031
Name: label2, dtype: int64

In [ ]:
def make_two_class(data_df):
  tag_list = []
  for i in range(len(data_df)):
      if data_df.label2[i] == 'no': 
        tag = "non-depressed"
        tag_list.append(tag)
      else: 
        tag = "depressed"      
        tag_list.append(tag)   
  data_df['label'] = tag_list 
  return data_df
new_df = make_two_class(data_df)

In [ ]:
new_df.tail(5)

##Two label droping

In [ ]:
def make_two_tag_droping(data_df):
  tag_list = []
  text_list = []
  for i in range(len(data_df)):
      if (data_df.class2[i] == 'anger' or data_df.class2[i] ==  'disapproval' or data_df.class2[i] == 'disgust'): 
        tag = "depressive"
        tag_list.append(tag)
        text_list.append(data_df.text[i])
      elif (data_df.class2[i] == 'fear' or data_df.class2[i] ==  'nervousness' or data_df.class2[i] ==  'embarrassment' or data_df.class2[i] ==  'remorse'): 
        tag = "depressive"
        tag_list.append(tag)
        text_list.append(data_df.text[i])
      elif (data_df.class2[i] == 'sadness' or data_df.class2[i] == 'disappointment'): 
        tag = "depressive"
        tag_list.append(tag)
        text_list.append(data_df.text[i])
      elif(data_df.class2[i] == 'joy' or data_df.class2[i] ==  'love' or data_df.class2[i] ==   'approval'or data_df.class2[i] ==  'amusement' ):
        tag = "non-depressive"      
        tag_list.append(tag)
        text_list.append(data_df.text[i])
  
  new_df = pd.DataFrame(list(zip(text_list, tag_list)), columns = ['text', 'label'])   

  

  return new_df
new_df = make_two_tag_droping(data_df)

In [ ]:
new_df.shape

(5452, 2)

In [ ]:
 or data_df.class2[i] ==  'neutral' 

#Droping Four class no, mild, moderate, and severe


In [ ]:
def make_four_label_droping(data_df):
  tag_list = []
  text_list = []
  for i in range(len(data_df)):
      if (data_df.class2[i] == 'anger' or data_df.class2[i] ==  'disapproval' or data_df.class2[i] == 'disgust'): 
        tag = "mild"
        tag_list.append(tag)
        text_list.append(data_df.text[i])
      elif (data_df.class2[i] == 'fear' or data_df.class2[i] ==  'nervousness' or data_df.class2[i] ==  'embarrassment' or data_df.class2[i] ==  'remorse'): 
        tag = "moderate"
        tag_list.append(tag)
        text_list.append(data_df.text[i])
      elif (data_df.class2[i] == 'sadness' or data_df.class2[i] == 'disappointment'): 
        tag = "severe"
        tag_list.append(tag)
        text_list.append(data_df.text[i])
      elif(data_df.class2[i] == 'joy' or data_df.class2[i] ==  'love' or data_df.class2[i] ==   'approval'or data_df.class2[i] ==  'amusement'  ):
        tag = "no"      
        tag_list.append(tag)
        text_list.append(data_df.text[i])
      else:
        pass
  
  new_df = pd.DataFrame(list(zip(text_list, tag_list)), columns = ['text', 'label'])   

  return new_df
new_df = make_four_label_droping(data_df)

In [ ]:
len(new_df.label)

5452

#Six Emotion

In [ ]:
def make_6_emotion(data_df):
  tag_list = []
  for i in range(len(data_df)):
      if (data_df.class2[i] == 'anger' or data_df.class2[i] ==  'disapproval' or data_df.class2[i] == 'disgust'): 
        tag = "anger"
        tag_list.append(tag)
      elif (data_df.class2[i] == 'fear' or data_df.class2[i] ==  'nervousness'): 
        tag = "fear"
        tag_list.append(tag)
      elif (data_df.class2[i] == 'sadness' or data_df.class2[i] == 'disappointment'or data_df.class2[i] ==  'embarrassment' or data_df.class2[i] ==  'remorse'): 
        tag = "sadness"
        tag_list.append(tag)
      elif (data_df.class2[i] == 'surprise' or data_df.class2[i] ==   'confusion'): 
        tag = "surprise"
        tag_list.append(tag)
      elif (data_df.class2[i] ==  'neutral'):

        tag = "neutral"
        tag_list.append(tag)
      else:
        tag = "joy"      
        tag_list.append(tag)   
  data_df['label'] = tag_list 
  return data_df
new_df = make_6_emotion(data_df)

In [ ]:
new_df.head(10)

In [ ]:
def make_7_emotion(data_df):
  tag_list = []
  for i in range(len(data_df)):
      if (data_df.class2[i] == 'anger' or data_df.class2[i] ==  'disapproval'): 
        tag = "anger"
        tag_list.append(tag)
      elif (data_df.class2[i] == 'disgust'):
        tag = "disgust"
        tag_list.append(tag)
      elif (data_df.class2[i] == 'fear' or data_df.class2[i] ==  'nervousness'): 
        tag = "fear"
        tag_list.append(tag)
      elif (data_df.class2[i] == 'sadness' or data_df.class2[i] == 'disappointment'or data_df.class2[i] ==  'embarrassment' or data_df.class2[i] ==  'remorse'): 
        tag = "sadness"
        tag_list.append(tag)
      elif (data_df.class2[i] == 'surprise' or data_df.class2[i] ==   'confusion'): 
        tag = "surprise"
        tag_list.append(tag)
      elif (data_df.class2[i] == 'neutral'): 
        tag = "neutral"
        tag_list.append(tag)
      else:
        tag = "joy"      
        tag_list.append(tag)   
  data_df['label'] = tag_list 
  return data_df

In [ ]:
new_df = make_7_emotion(data_df)

In [ ]:
text_data = new_df["text"]
labels = new_df["label"]

In [ ]:
labels.value_counts()

depressive        3629
non-depressive    1823
Name: label, dtype: int64

In [ ]:
set(labels)

{'mild', 'moderate', 'no', 'severe'}

In [ ]:
from sklearn.preprocessing import LabelEncoder
label = LabelEncoder()
labels = label.fit_transform(labels)

## train test split scikit-learn

In [ ]:

import numpy as np
from sklearn.model_selection import train_test_split
train_data, test_data, train_labels, test_labels = train_test_split(text_data, labels, test_size=0.2, shuffle = True, random_state=42)

In [ ]:
train_labels.value_counts()

severe      1675
no          1468
mild         740
moderate     478
Name: label, dtype: int64

In [ ]:
test_labels.value_counts()

severe      418
no          355
mild        197
moderate    121
Name: label, dtype: int64

In [ ]:
text_clf = Pipeline([
        ('FE', TfidfVectorizer(ngram_range=(1,2), use_idf = True, tokenizer=lambda x: x.split(), max_features=10000)),  
        ('Transformer', TfidfTransformer()),
       ('clf',SVC(class_weight='balanced')),
])

text_clf.fit(train_data, train_labels)
predicted = text_clf.predict(test_data)
import pandas
report = classification_report(test_labels, predicted, output_dict=True)
df = pandas.DataFrame(report).transpose().round(decimals=2)
df.to_csv('/content/drive/MyDrive/Thesis/Two Level Result/check/6svm_report.csv')
report1 = confusion_matrix(test_labels, predicted)
df = pandas.DataFrame(report1)
df.to_csv('/content/drive/MyDrive/Thesis/Two Level Result/check/6svm_confusion.csv')


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
%%time
lr_model = LogisticRegressionCV(class_weight='balanced', max_iter=10000)
dt_model = DecisionTreeClassifier(class_weight='balanced')
rf_model = RandomForestClassifier(class_weight='balanced')

mnb_model = MultinomialNB()
svm_model = SVC(class_weight='balanced')
knn_model = KNeighborsClassifier()
ada_model = AdaBoostClassifier()

model_names = ['Logistic Regression','Decision Tree','Random Forest','Naive Bayes', 'SVM', 'KNN', 'AdaBoost']
ml_models = [lr_model,dt_model,rf_model,mnb_model, svm_model, knn_model, ada_model]

CPU times: user 248 µs, sys: 0 ns, total: 248 µs
Wall time: 1.97 ms


In [ ]:
%time
def model_train(model, train_data, train_labels):
  pipe = Pipeline([
        ('FE', TfidfVectorizer(ngram_range=(1,1), use_idf = True, tokenizer=lambda x: x.split(), max_features=150000)),  
        ('Transformer', TfidfTransformer()),  
        ('M', model)
    ])
  pipe.fit(train_data, train_labels)
  return pipe

trained_ml_models = []
for i in ml_models:
    tt = model_train(i,train_data, train_labels)
    trained_ml_models.append(tt)
pr_list = []
re_list = []
f1_list = []

def print_results(md, x_test, y_test, name):
    #print("\n\nClassifier: ", name)
    pred_y = md.predict(x_test)
    true, pred = y_test, pred_y
    # print(confusion_matrix(true,pred))
    # print(classification_report(true,pred))
    # print("\nPrecison : ",precision_score(pred,true, average = 'weighted'))
    # print("\nRecall : ",recall_score(pred,true,  average = 'weighted'))
    # print("\nF1 : ",f1_score(pred,true,  average = 'weighted'))

    pr_list.append((round(precision_score(y_test, pred_y, average='weighted'),4)*100)) 
    re_list.append((round(recall_score(y_test, pred_y, average='weighted'),4)*100))
    f1_list.append((round(f1_score(y_test, pred_y, average='weighted'),4)*100))

for i in range(len(trained_ml_models)):
    md = trained_ml_models[i]
    name = model_names[i]
    print_results(md, test_data, test_labels, name)

performance_matrix = pd.DataFrame({'Precision':pr_list,
                                  'Recall':re_list,'F1 Score':f1_list},
                                 index =model_names)
performance_matrix.to_csv('/content/drive/MyDrive/Thesis/Two Level Result/check/15k_shuffle.2Test_drop.csv')

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 10.3 µs


In [ ]:

performance_matrix = pd.DataFrame({'Precision':pr_list,
                                   'Recall':re_list,'F1 Score':f1_list},
                                  index =model_names)
performance_matrix.to_csv('/content/drive/MyDrive/Thesis/Thesis Code/thesis result/10k_.2Test_drop.csv')

In [ ]:
train_labels.value_counts()

severe      1674
no          1468
mild         740
moderate     479
Name: label, dtype: int64

In [ ]:
text_clf = Pipeline([
        ('FE', TfidfVectorizer(ngram_range=(1,2), use_idf = True, tokenizer=lambda x: x.split(), max_features=15000)),  
        ('Transformer', TfidfTransformer()),
       ('clf',LogisticRegressionCV(class_weight='balanced', max_iter=5000)),
])

text_clf.fit(train_data, train_labels)
predicted = text_clf.predict(test_data)
import pandas
report = classification_report(test_labels, predicted, output_dict=True)
df = pandas.DataFrame(report).transpose().round(decimals=2)
df.to_csv('/content/drive/MyDrive/Thesis/Two Level Result/check/report_2lrdrop.csv')
report1 = confusion_matrix(test_labels, predicted)
df = pandas.DataFrame(report1)
df.to_csv('/content/drive/MyDrive/Thesis/Two Level Result/check/confusion_2lr_drop.csv')


## Ensemble Train, and Test

##Make Whole list

In [ ]:
whole_data = []

def make_whole_list(L):
  for i in L:
    whole_data.append(i)

make_whole_list(train_data)
make_whole_list(test_data)
print(len(whole_data))

2702


In [ ]:
tf = TfidfVectorizer(ngram_range=(1,2), use_idf = True, tokenizer=lambda x: x.split(),  max_features=15000)
tf.fit_transform(whole_data) ### Bit confusion arises here need to resolve
trained_texts = tf.transform(train_data)
test_texts = tf.transform(test_data)

In [ ]:
%%time
from sklearn.ensemble import VotingClassifier
LogReg_clf = LogisticRegressionCV(class_weight='balanced', max_iter=400)
#rt_clf =   RandomForestClassifier(class_weight='balanced')
SVC_clf = SVC(class_weight='balanced')
#ada_model = AdaBoostClassifier()
voting_clf = VotingClassifier(estimators=[ ('LogReg', LogReg_clf),('SVC', SVC_clf)], voting='hard')
voting_clf.fit(trained_texts, train_labels)


CPU times: user 13.7 s, sys: 14.8 s, total: 28.5 s
Wall time: 15.2 s


## Test 

In [ ]:
preds = voting_clf.predict(test_texts)
true, pred = test_labels, preds
print(confusion_matrix(true,pred))
print(classification_report(true,pred))
print("\nPrecison : ",precision_score(true, pred, average = 'weighted'))
print("\nRecall : ",recall_score(true, pred,  average = 'weighted'))
print("\nF1 : ",f1_score(true, pred,  average = 'weighted'))

[[422   2]
 [115   2]]
                precision    recall  f1-score   support

    depressive       0.79      1.00      0.88       424
non-depressive       0.50      0.02      0.03       117

      accuracy                           0.78       541
     macro avg       0.64      0.51      0.46       541
  weighted avg       0.72      0.78      0.70       541


Precison :  0.7240281980056245

Recall :  0.7837338262476895

F1 :  0.6954649550969024


## Optimization with optuna

In [ ]:
X_train = train_data
y_train = train_labels

In [ ]:
!pip install optuna

In [ ]:
import optuna

### Logistic Regression

In [ ]:
def objective_lr(trial):

    params = { 
        'class_weight': trial.suggest_categorical('class_weight', ['balanced', None]),
        'solver': trial.suggest_categorical('solver', ['newton-cg', 'sag', 'saga', 'lbfgs']),
        'multi_class':'multinomial',
        'max_iter': 5000
        }
    text_clf = Pipeline([
        ('FE', TfidfVectorizer(ngram_range=(1,1), use_idf = True, tokenizer=lambda x: x.split(), max_features=15000)),  
        ('Transformer', TfidfTransformer()),
       ( 'clf' ,LogisticRegression(**params, random_state = 22).set_params(**params))  
    
    
])

    text_clf.fit(train_data, train_labels)
    


    return np.mean(cross_val_score(text_clf,train_data, train_labels,cv =  scoring='accuracy'))

In [ ]:
study = optuna.create_study(direction='maximize') #opimize rapidly
study.optimize(objective_lr, n_trials=10)

[I 2022-03-24 04:29:56,896] A new study created in memory with name: no-name-cfd30abc-e6c4-4706-a543-0b788f7f3ed5
[I 2022-03-24 04:30:00,928] Trial 0 finished with value: 0.6986659924390185 and parameters: {'class_weight': None, 'solver': 'lbfgs'}. Best is trial 0 with value: 0.6986659924390185.
[I 2022-03-24 04:30:03,712] Trial 1 finished with value: 0.6986659924390185 and parameters: {'class_weight': None, 'solver': 'newton-cg'}. Best is trial 0 with value: 0.6986659924390185.
[I 2022-03-24 04:30:08,243] Trial 2 finished with value: 0.6986659924390185 and parameters: {'class_weight': None, 'solver': 'lbfgs'}. Best is trial 0 with value: 0.6986659924390185.
[I 2022-03-24 04:30:12,265] Trial 3 finished with value: 0.6986659924390185 and parameters: {'class_weight': None, 'solver': 'lbfgs'}. Best is trial 0 with value: 0.6986659924390185.
[I 2022-03-24 04:30:14,535] Trial 4 finished with value: 0.6984028372600214 and parameters: {'class_weight': 'balanced', 'solver': 'sag'}. Best is tri

In [ ]:
def objective_mlp(trial):

    params = {
        'activation': trial.suggest_categorical('activation', ['logistic', 'tanh', 'relu']),
        'solver': trial.suggest_categorical('solver', ['lbfgs', 'sgd', 'adam']),
        'hidden_layer_sizes':trial.suggest_int('hidden_layer_sizes', 100, 1500),
        'alpha': trial.suggest_uniform('alpha', 0.001, 0.99),
        'batch_size':trial.suggest_int('batch_size', 150, 300), 
        'learning_rate': trial.suggest_categorical('learning_rate', ['adaptive', 'constant', 'invscaling']),
        'max_iter': 1000
        }
  
    model = MLPClassifier(**params, random_state = 22) 
    
    model.set_params(**params)

    return np.mean(cross_val_score(model, train_data, train_labels, cv=5, scoring='accuracy'))

In [ ]:
study = optuna.create_study(direction='maximize') #optimizattion required huge time
study.optimize(objective_mlp, n_trials=15)

[I 2021-09-18 15:13:57,761] A new study created in memory with name: no-name-4dd592fc-e8e8-4012-bfb1-48fe10d1f351


In [ ]:
def objective_rf(trial):
    
    params = {
        'criterion': trial.suggest_categorical('criterion', ['gini', 'entropy']),
        'max_depth': trial.suggest_int('max_depth', 5, 25),
        'n_estimators': trial.suggest_int('n_estimators', 15000, 25000),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 2, 5),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 25),
        'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 50, 250),
        }
    text_clf = Pipeline([
        ('FE', TfidfVectorizer(ngram_range=(1,2), use_idf = True, tokenizer=lambda x: x.split(), max_features=1000)),  
        ('Transformer', TfidfTransformer()),
       ( 'clf' ,RandomForestClassifier(**params, random_state = 22).set_params(**params))  
    
    
])

    text_clf.fit(train_data, train_labels)
   
 

    return np.mean(cross_val_score(text_clf, X_train, y_train, cv=5, scoring='accuracy'))


In [ ]:
study = optuna.create_study(direction='maximize') #optimization  required much time
study.optimize(objective_rf, n_trials=5) 

[I 2022-03-26 03:22:17,309] A new study created in memory with name: no-name-2824d2b6-2a11-4c5d-b6f5-7de4823514e7
[I 2022-03-26 03:32:00,635] Trial 0 finished with value: 0.5814932761783156 and parameters: {'criterion': 'entropy', 'max_depth': 13, 'n_estimators': 20545, 'min_samples_leaf': 3, 'min_samples_split': 6, 'max_leaf_nodes': 99}. Best is trial 0 with value: 0.5814932761783156.
[I 2022-03-26 03:38:56,610] Trial 1 finished with value: 0.5830695994081821 and parameters: {'criterion': 'gini', 'max_depth': 12, 'n_estimators': 19253, 'min_samples_leaf': 2, 'min_samples_split': 13, 'max_leaf_nodes': 112}. Best is trial 1 with value: 0.5830695994081821.
[I 2022-03-26 03:45:42,470] Trial 2 finished with value: 0.5704640805034507 and parameters: {'criterion': 'entropy', 'max_depth': 6, 'n_estimators': 20904, 'min_samples_leaf': 2, 'min_samples_split': 22, 'max_leaf_nodes': 88}. Best is trial 1 with value: 0.5830695994081821.
[I 2022-03-26 03:50:48,808] Trial 3 finished with value: 0.570